In [1]:
import cv2
import torch
torch.cuda.is_available()

True

In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.83 🚀 Python-3.10.11 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1070, 8114MiB)
Setup complete ✅ (12 CPUs, 15.5 GB RAM, 119.0/467.9 GB disk)


In [3]:
torch.cuda.device_count()

1

In [4]:
from ultralytics import YOLO

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("working on gpu")
else:
    device = torch.device("cpu")
    print("working on cpu")

working on gpu


In [6]:
def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))

In [7]:
force_cudnn_initialization()

In [8]:
model = YOLO("yolov8l-cls.pt")

results = model.train(data='/home/ira-demo/EEfinal_rasp/datasets/dog-pose-yolo_dataset', epochs=100, patience=5, imgsz=224, plots=True)

100%|██████████| 71.7M/71.7M [00:03<00:00, 23.3MB/s]
New https://pypi.org/project/ultralytics/8.0.86 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.83 🚀 Python-3.10.11 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1070, 8114MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8l-cls.pt, data=/home/ira-demo/EEfinal_rasp/datasets/dog-pose-yolo_dataset, epochs=100, patience=5, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

In [9]:
# Evaluate the model's performance on the validation set
results = model.val()

Ultralytics YOLOv8.0.83 🚀 Python-3.10.11 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1070, 8114MiB)
YOLOv8l-cls summary (fused): 133 layers, 36188419 parameters, 0 gradients, 98.7 GFLOPs


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [17]:
# Perform object detection on an image using the model
results = model.predict(source='../Picture/vlc_8TgxGDhtjf.jpg', save=True)


image 1/1 /home/ira-demo/EEfinal_rasp/Picture/vlc_8TgxGDhtjf.jpg: 224x224 Lying 0.82, Standing 0.18, Sitting 0.00, 8.4ms
Speed: 14.9ms preprocess, 8.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Results saved to /home/ira-demo/EEfinal_rasp/runs/classify/predict2


In [36]:
results[0]

ultralytics.yolo.engine.results.Results object with attributes:

boxes: None
keypoints: None
keys: ['probs']
masks: None
names: {0: 'Lying', 1: 'Sitting', 2: 'Standing'}
orig_img: array([[[56, 92, 92],
        [57, 93, 93],
        [56, 95, 93],
        ...,
        [ 4, 60, 41],
        [ 3, 59, 40],
        [ 3, 59, 40]],

       [[58, 94, 94],
        [59, 95, 95],
        [58, 97, 95],
        ...,
        [ 4, 60, 41],
        [ 3, 59, 40],
        [ 3, 59, 40]],

       [[59, 97, 97],
        [60, 98, 98],
        [60, 99, 97],
        ...,
        [ 3, 59, 40],
        [ 2, 58, 39],
        [ 2, 58, 39]],

       ...,

       [[ 1, 38,  6],
        [ 2, 39,  7],
        [ 1, 40,  8],
        ...,
        [ 3, 65, 25],
        [ 3, 60, 21],
        [ 0, 57, 18]],

       [[ 2, 39,  7],
        [ 3, 40,  8],
        [ 1, 40,  8],
        ...,
        [ 5, 68, 28],
        [ 3, 63, 23],
        [ 0, 59, 19]],

       [[ 3, 40,  8],
        [ 3, 40,  8],
        [ 2, 41,  9],
      

In [40]:
print(results[0].probs)
print(results[0].probs.cpu())

tensor([0.8209, 0.0026, 0.1765], device='cuda:0')
tensor([0.8209, 0.0026, 0.1765])


In [11]:
video_path = "/home/ira-demo/Videos/sitting_dog_vid.mp4"
cap = cv2.VideoCapture(video_path)
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference on the frame
        results = model(frame)
        print("Printing the result")
        print("boxes:")
        print(results[0].boxes)
        print("probs:")
        print(results[0].probs)
        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.1ms
Speed: 0.2ms preprocess, 7.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


Printing the result
boxes:
None
probs:
tensor([9.9995e-01, 3.0456e-06, 4.3278e-05], device='cuda:0')



0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.3ms
Speed: 0.3ms preprocess, 7.3ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 9.0ms
Speed: 0.3ms preprocess, 9.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.0ms
Speed: 0.3ms preprocess, 7.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.5ms
Speed: 0.2ms preprocess, 7.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.7ms
Speed: 0.2ms preprocess, 6.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.6ms
Speed: 0.2ms preprocess, 6.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


Printing the result
boxes:
None
probs:
tensor([9.9996e-01, 2.7800e-06, 3.8379e-05], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9997e-01, 1.9430e-06, 2.9197e-05], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9997e-01, 1.5120e-06, 2.9079e-05], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9998e-01, 1.4953e-06, 2.0377e-05], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9998e-01, 1.9296e-06, 2.0301e-05], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9998e-01, 2.1199e-06, 2.0465e-05], device='cuda:0')



0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.7ms
Speed: 0.3ms preprocess, 6.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.8ms
Speed: 0.3ms preprocess, 6.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.8ms
Speed: 0.3ms preprocess, 6.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.1ms
Speed: 0.2ms preprocess, 7.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 6.9ms
Speed: 0.2ms preprocess, 6.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.3ms
Speed: 0.3ms preprocess, 7.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Standing 0.00, Sitting 0.00, 7.3ms
Speed: 0.2m

...

Printing the result
boxes:
None
probs:
tensor([9.9882e-01, 9.6763e-04, 2.1132e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9887e-01, 8.8277e-04, 2.4919e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9874e-01, 9.9337e-04, 2.6795e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9879e-01, 9.6123e-04, 2.5031e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9877e-01, 9.9623e-04, 2.3606e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9866e-01, 1.1292e-03, 2.0773e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9861e-01, 1.1903e-03, 1.9846e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9887e-01, 9.0395e-04, 2.2617e-04], device='cuda:0')


0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 7.0ms
Speed: 0.3ms preprocess, 7.0ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 7.6ms
Speed: 0.3ms preprocess, 7.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 6.4ms
Speed: 0.3ms preprocess, 6.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 6.8ms
Speed: 0.3ms preprocess, 6.8ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 7.9ms
Speed: 0.3ms preprocess, 7.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 6.7ms
Speed: 0.3ms preprocess, 6.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 7.1ms
Speed: 0.3ms

Printing the result
boxes:
None
probs:
tensor([9.9909e-01, 7.0204e-04, 2.0471e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9883e-01, 9.4931e-04, 2.2493e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9884e-01, 9.6469e-04, 1.9320e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9863e-01, 1.1535e-03, 2.1796e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9838e-01, 1.3802e-03, 2.3619e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9786e-01, 1.8202e-03, 3.1943e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9775e-01, 1.9247e-03, 3.2734e-04], device='cuda:0')
Printing the result
boxes:
None
probs:
tensor([9.9779e-01, 1.9506e-03, 2.5655e-04], device='cuda:0')



0: 224x224 Lying 1.00, Sitting 0.00, Standing 0.00, 6.3ms
Speed: 0.3ms preprocess, 6.3ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


Printing the result
boxes:
None
probs:
tensor([9.9769e-01, 2.0671e-03, 2.4693e-04], device='cuda:0')
